In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
import math
import time
from PIL import Image
from imageai.Detection import ObjectDetection

Using TensorFlow backend.


In [2]:
model_path = './models/'
model_name = 'yolo.h5'
model = model_path + model_name

In [3]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(model)
detector.loadModel()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
input_path = './input/images/'
output_path = './output/images/'
result_path = './output/result/'

In [5]:
file_name = 'crowd.jpg'

In [6]:
input_file = input_path + file_name
model = model_path + model_name
output_file = output_path + file_name 
result_file = result_path + file_name.split('.')[0] + '.txt'

In [7]:
image = tf.keras.preprocessing.image.load_img(input_file)
image = tf.keras.preprocessing.image.img_to_array(image)

In [8]:
shape = image.shape
height = shape[0]
width = shape[1]

In [9]:
offset_height = 0.05 * height
offset_width = 0.03 * width

In [10]:
# # number of vertical cuts
# vert_pieces = math.ceil(height/416)
# # number of horizontal cuts
# horiz_pieces = math.ceil(width/416)
vert_pieces = 2
horiz_pieces = 2

In [11]:
crop_height = int(math.ceil(height/vert_pieces + offset_height))
crop_width = int(math.ceil(width/horiz_pieces + offset_width))

In [12]:
boxes = []
x1 = 0
y1 = 0
for i in range(vert_pieces):
    y = y1 + crop_height/height
    for j in range(horiz_pieces):
        x = x1 + crop_width/width
        if(y > 1):
            y = 1
        if(x > 1):
            x = 1
        boxes.append([y1,x1,y,x])
        x1 = x - (2*offset_width)/width
    x1 = 0
    y1 = y - (2*offset_height)/height
boxes = np.array(boxes)

In [13]:
images = []
images.append(image)
images = np.array(images)
num_boxes = boxes.shape[0]
box_indices = np.zeros(num_boxes)

In [14]:
crop_tensor = tf.image.crop_and_resize(images, boxes, box_indices, crop_size=[crop_height,crop_width], method='bilinear', extrapolation_value=0, name=None)

cropped_images = crop_tensor.eval(session=tf.Session())

Instructions for updating:
box_ind is deprecated, use box_indices instead


In [15]:
tic = time.time()

crop_detections = []
for i in range(num_boxes):
    temp_img, crop_det = detector.detectObjectsFromImage(input_image=cropped_images[i],input_type="array", output_type="array",minimum_percentage_probability=50)
#     crop_det = detector.detectObjectsFromImage(input_image=cropped_images[i],input_type="array", output_image_path="crop_det"+str(i)+".jpg", minimum_percentage_probability=50)
    crop_detections.append(crop_det)

tac = time.time()

elapsed = tac-tic
print(elapsed)

11.676269054412842


In [16]:
detected_boxes = []
detections = []
scores = []
for i in range(num_boxes):
    size = len(crop_detections[i])
    x = boxes[i][1]*width
    y = boxes[i][0]*height
    for j in range(size):
        detected = crop_detections[i][j]
        box = detected['box_points']
        box = [box[0]+x, box[1]+y, box[2]+x, box[3]+y]
        detected_boxes.append(box)
        detected['box_points'] = box
        scores.append(detected['percentage_probability'])
        detections.append(detected)
detected_boxes = np.array(detected_boxes)
scores = np.array(scores)

In [18]:
selected_indices = tf.image.non_max_suppression(tf.convert_to_tensor(detected_boxes, dtype=tf.float32), tf.convert_to_tensor(scores, dtype=tf.float32), len(detected_boxes), 0.5)
selected_indices = selected_indices.eval(session=tf.Session())

In [ ]:
final_detections = []
for index in selected_indices:
    final_detections.append(detections[index])